<a href="https://colab.research.google.com/github/Dhanyahree04/AI-resume-ranking/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install streamlit pyngrok PyPDF2 python-docx scikit-learn spacy
!python -m spacy download en_core_web_sm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
%%writefile app.py
import streamlit as st
import PyPDF2
import docx
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load SpaCy Model
nlp = spacy.load("en_core_web_sm")

# Function to extract text from PDF/DOCX
def extract_text(file):
    text = ""
    if file.name.endswith(".pdf"):
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + " "
    elif file.name.endswith(".docx"):
        doc = docx.Document(file)
        for para in doc.paragraphs:
            text += para.text + " "
    return text.strip()

# Function to preprocess text
def preprocess_text(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

# Streamlit UI
st.title("📄 AI Resume Screening & Ranking System")
st.write("Upload resumes and provide a job description to rank candidates.")

uploaded_files = st.file_uploader("Upload Resume Files (PDF/DOCX)", accept_multiple_files=True)
job_description = st.text_area("Enter Job Description", "")

if st.button("Rank Candidates"):
    if not uploaded_files or not job_description.strip():
        st.error("Please upload resumes and enter a job description.")
    else:
        processed_jd = preprocess_text(job_description)
        candidates = {}

        for file in uploaded_files:
            resume_text = extract_text(file)
            processed_resume = preprocess_text(resume_text)

            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform([processed_jd, processed_resume])
            similarity_score = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

            candidates[file.name] = similarity_score

        ranked_candidates = sorted(candidates.items(), key=lambda x: x[1], reverse=True)

        st.subheader("📊 Ranked Candidates")
        for i, (candidate, score) in enumerate(ranked_candidates, 1):
            st.write(f"**{i}. {candidate}** - Score: {score:.2f}")


Overwriting app.py


In [ ]:
!pip install pyngrok
import os
import threading
import time
from pyngrok import ngrok

# Start Streamlit in a separate thread
def start_streamlit():
    os.system("streamlit run app.py --server.port 8501")

thread = threading.Thread(target=start_streamlit)
thread.start()

time.sleep(5)  # Wait for Streamlit to start

# Set up ngrok
ngrok.set_auth_token("2ucHGXgsBCOLijgybTQQVhH61NJ_5WqxWUZzNxVjNcFGqN1EN")  # Replace with your new ngrok token
public_url = ngrok.connect(8501, bind_tls=True).public_url
print(f"🚀 Streamlit is live at: {public_url}")

🚀 Streamlit is live at: https://8f1c-35-227-115-80.ngrok-free.app
